### **Ujian Akhir Semester Analitika Media Sosial**

Nama : Faustina Maureen Widjojo

NIM : A12.2020.06526

Kelompok : A12.6501

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# import sys
# sys.path.append('/content/drive/My Drive')

The pathlib is a Python module which provides an object API for working with files and directories.

In [61]:
import pathlib
pathlib.Path().resolve()

PosixPath('/content')

NLTK consists of the most common algorithms such as tokenizing, part-of-speech tagging, stemming, sentiment analysis, topic segmentation, and named entity recognition.

NLTK helps the computer to analysis, preprocess, and understand the written text.

In [62]:
#importing the required packages
import nltk
#### Run it, if before never download nltk library
# nltk.download('all')
import re
import string
import gensim 
from gensim.parsing.preprocessing import remove_stopwords
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from nltk import pos_tag,word_tokenize,ne_chunk
from sklearn.feature_extraction.text import TfidfVectorizer
from pandas import DataFrame

from nltk.corpus import wordnet,stopwords
import spacy
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

**Uploading Reference Article Cholera.txt**


In [75]:
from google.colab import files
# uploaded=files.upload()
sample = open("Cholera.txt", "r") 

s = sample.read()

In [76]:
s

'Cholera is an infection of the small intestine by some strains of the bacterium Vibrio cholerae. Symptoms may range from none, to mild, to severe. The classic symptom is large amounts of watery diarrhea that lasts a few days. Vomiting and muscle cramps may also occur. Diarrhea can be so severe that it leads within hours to severe dehydration and electrolyte imbalance. This may result in sunken eyes, cold skin, decreased skin elasticity, and wrinkling of the hands and feet. Dehydration can cause the skin to turn bluish. Symptoms start two hours to five days after exposure.\nCholera is caused by a number of types of Vibrio cholerae, with some types producing more severe disease than others. Cholera is spread mostly by unsafe water and unsafe food that has been contaminated with human feces containing the bacteria. Undercooked shellfish is a common source. Humans are the only known host for the bacteria. Risk factors for the disease include poor sanitation, not enough clean drinking wate

**Text Preprocessing Functions**

Stemming

Stopword

create_document_term_matrix: Function to create TF - IDF Matrix

calculate_cosine_similarity: Function to return the top 3 sentences based on cosine similarity between TF-IDF scores

In [65]:
def stem_sentence(sentence):
  words=word_tokenize(sentence)
  #lemmatizer = WordNetLemmatizer()
  
  
  stemmer = SnowballStemmer("english")
  new_words=[]
  for i in words:
    new_words.append(stemmer.stem(i))
    new_words.append(" ")
  return "".join(new_words)  
  


def clean_sentence(sentence, stopwords=True):
    
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    
    
    
    if stopwords:
         sentence = remove_stopwords(sentence)
    
   
    
    return sentence
def get_cleaned_sentences(sents,stopwords=True):    
    
    cleaned_sentences=[]

    for i in sents:
        
        cleaning=clean_sentence(i,stopwords)
        cleaned=stem_sentence(cleaning)
        cleaned_sentences.append(cleaned)
    return cleaned_sentences

In [66]:
def create_document_term_matrix(sen,vectorizer):
  doc_term_matrix=vectorizer.fit_transform(sen)
  return DataFrame(doc_term_matrix.toarray(), columns=vectorizer.get_feature_names())
  

In [67]:
def calculate_cosine_similarity(df_list,sentences,question):
  a=[]
  for i in range(len(df_list)-1):
    sim=1 - spatial.distance.cosine(df_list[i], question)
    t=(sim,sentences[i])
    a.append(t)
  a.sort(reverse=True)
  n=[]
  for i in range(3):
    n.append(a[i][1])
    #print("*",n[i])
   
    
  return n 

Function to classify question

In [68]:
def questiontype( question):
        questiontags = ['WP','WDT','WP$','WRB']
        question_POS = pos_tag(word_tokenize(question.lower()))
        
        question_Tags=[]
        for token in question_POS:
            if token[1] in questiontags:
              question_Tags.append(token)
                
                
        if len(question_Tags)==1 and question_Tags[0][0]!= 'what' :
          return True
        else:
          return False  

Function to find the most relevan sentence using n-gram similarity

In [69]:
from nltk.util import ngrams
def n_gram_similarity(question,n):
  q=list(ngrams(word_tokenize(question.lower()),1))
  a=0
  b=0
  c=0
  t=[]
  for i in q:
    if i in list(ngrams(word_tokenize(n[0].lower()),1)):
      a=a+1
  for i in q:
    if i in list(ngrams(word_tokenize(n[1].lower()),1)):
      b=b+1
  for i in q:
    if i in list(ngrams(word_tokenize(n[2].lower()),1)):
      c=c+1        
  d=max(a,b,c)
  if a == d:
    t.append(n[0])
  if b == d:
    t.append(n[1]) 
  if c ==d:
    t.append(n[2])
  print()  
  #print("Selected Sentence:",t[0])  
  return t

answertype : Determines the type of entity that has to be returned, performs entity ranking if multiplr entitles a present

In [70]:
def answertype(question):
  nlp = spacy.load('en_core_web_sm')

  if (questiontype(question)):
    t='DESCRIPTIVE'
    flag=0
    word=word_tokenize(question.lower())
  
    if 'who' in word:
      t='PERSON'
    elif 'where' in word:
      t='GPE'
    elif 'how' in word and 'many' in word and  'age' in word or 'duration' in word or 'long' in word or 'days'in word or 'years' in word or'months' in word:
      t='DATE' 
    elif 'how' in word and 'many' in word :
       t = 'CARDINAL'  
    elif 'when' in word  or 'age' in word or 'period' in word or 'duration' in word  or 'old' in word or 'long' in word:
      t='DATE'
    elif 'how' in word  and 'long' in word or 'often' or 'age' in word or 'years' in word:
      t='DATE' 
    elif 'what' in word and 'time' in word or 'duration' in word or 'period' in  'word'  :
      t='DATE' 
    i=len(df_list)-1  
    n=calculate_cosine_similarity(df_list, sentences,df_list[i])
    n=n_gram_similarity(question,n)
    #print("Most relevant sentence", n[0])
    #print("ANSWER TYPE:",t)
    key = n[0]
    spdoc = nlp(key)
    entity_type=[]
    for ent in spdoc.ents:
       if ent.label_ == t:
          entity_type.append(ent.text)
    if len(entity_type) == 1:
      #print("ANSWER TYPE:", t)
      print("ANSWER:", entity_type[0])  
    if len(entity_type) == 0:
      #print("ANSWER TYPE:", t) 
      print(n[0])
    if len(entity_type) > 1:
      #print("Answer Type:",t)  
      key_question = question
      q=[]
      spdoc = nlp(key_question)
      for ent in spdoc:
        if ent.pos_ == 'NOUN' or ent.pos_ =='ADJ' :
          q.append(ent.text)
  
      key_answer = n[0]
      a = []
      spd = nlp(key)
      for ent in spd:
        if ent.pos_ == 'NOUN'or ent.pos_ =='ADJ' :
          a.append(ent.text)
  #s=[sentence.index(i) for i in t]
      s=[]
      w=[]
      for i in entity_type:
       s.append(n[0].index(i))
      for i in range(len(s)):
        w.append(0)

    
      for i in q:
        try:
           factor= n[0].index(i)
           for j in range(len(s)):
              w[j]=w[j]+(abs(s[j]-factor))
        except:
           continue    
      m=min(w)
      u=[]
      for i in range(len(s)):
        if w[i] == m:
           #print(entity_type[i])
           u.append(entity_type[i])
      print("ANSWER:",u[0])     
      

  

    
  else:
    t='DESCRIPTIVE'
    #print("ANSWER TYPE:",t)
    i=len(df_list)-1  
    n=calculate_cosine_similarity(df_list, sentences,df_list[i])
    #n = n_gram_similarity(question, n)
    for j in n:
         print(j) 

In [73]:
#from textblob import TextBlob

question=["What is Cholera?", "What is the symptom of cholera?", "How long do the symptoms start after exposure?", "How is Cholera spread mostly?", "What is the common source of Vibrio cholerae?", "What are some risk factors for the disease?", "How can cholera be diagnosed?", "How are the prevention methods againts cholera?", "How long is the protection from the cholera vaccines?", "What is the primary treatment for affected individual?", "How many death does cholera affect?", "What year is the most recent cholera pandemics started?", "What is the primary symptoms of cholera?", "Why is cholera nicknamed the 'blue death'?"]
for j in question:
  #j=TextBlob(j)
  #j=str(j.correct())
 
  qq=[]
  qp=[]
  que=sent_tokenize(j)
  
  qq.append(que)
  qp.append(j)
  questiontags = ['WP','WDT','WP$','WRB']
  question_POS = pos_tag(word_tokenize(j.lower()))
        
  question_Tags=[]
  for token in question_POS:
      if token[1] in questiontags:
          question_Tags.append(token)
  
 
  if len(question_Tags)>1:
     if ' and ' in j :
       pos=j.lower().find('and')
       qq=[]
       qp=[]
       qp.append(j[:pos])
       qp.append(j[pos+1:])
       qq.append(sent_tokenize(j[:pos])) 
       qq.append(sent_tokenize(j[pos+1:])) 
  
     
               
  
  print("QUESTION:",j)
  for k in range(len(qp)):   
    
    sentences=sent_tokenize(s)
    #q contains a list of cleaned sentence tokens of question
    q=get_cleaned_sentences(qq[k],stopwords=True)
    #preprocessed contains a list of cleaned sentence tokens of the reference text
    preprocessed=get_cleaned_sentences(sentences,stopwords=True)
  
  preprocessed.append(q[0])
  i=len(preprocessed)-1
    
  
  tfidf_vect=TfidfVectorizer()
  df=create_document_term_matrix(preprocessed,tfidf_vect) 
  df_list = df.values.tolist()
  answertype(qp[k])
  print()

QUESTION: What is Cholera?
Cholera is an infection of the small intestine by some strains of the bacterium Vibrio cholerae.
Cholera is caused by a number of types of Vibrio cholerae, with some types producing more severe disease than others.
Cholera bacteria have been found in shellfish and plankton.

QUESTION: What is the symptom of cholera?
Symptoms may range from none, to mild, to severe.
The primary symptoms of cholera are profuse diarrhea and vomiting of clear fluid.
Symptoms start two hours to five days after exposure.

QUESTION: How long do the symptoms start after exposure?

Symptoms start two hours to five days after exposure.

QUESTION: How is Cholera spread mostly?

Cholera is spread mostly by unsafe water and unsafe food that has been contaminated with human feces containing the bacteria.

QUESTION: What is the common source of Vibrio cholerae?
Undercooked shellfish is a common source.
Cholera is an infection of the small intestine by some strains of the bacterium Vibrio ch